# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [132]:
import re
import os


In [143]:
def generate_lt_regex(value):
    digits = list(str(value))  # Split value into individual digits
    print(digits)
    n = len(digits)  # Length of the number
    regex =""
    if int(digits[0]) > 1:
        regex += f"(^0*[1-{int(digits[0])-1}][0-9]{{{n-1}}}$)"
    else:
        pass
        regex +=f"{value-1}"
        # for i in range(1, n-1):
        #     regex += f"(^0*1" + "".join(digits[1:i]) + f"[0-{int(digits[i])-1}][0-9]{{{n-1-i}}}$)|"
    for i in range(1, n):
        regex += f"|(^0*" + "".join(digits[:i]) + f"[0-{int(digits[i])-1}][0-9]{{{n-1-i}}}$)"
    regex += f"|(^0*"+ f"[0-9]{{1,{n-1}}}$)"

    return regex


print(generate_lt_regex(123))
def generate_eq_regex(value, is_not=False):
    """Tạo regex cho phép toán = value"""
    return f"(^0*{value}$)" if not is_not else f"(^(?!0*{value}$).+)"

def generate_le_regex(value):
    """Tạo regex cho phép toán ≤ value"""
    return generate_lt_regex(value + 1)

print(generate_le_regex(123))

def generate_ge_regex(value):
    """Tạo regex cho phép toán ≥ value (phủ định của < value)"""
    return f"^(?!({generate_lt_regex(value)})).*"
print(generate_ge_regex(123))
def generate_gt_regex(value):
    """Tạo regex cho phép toán > value (phủ định của ≤ value)"""
    leng=len(str(value))
    return f"(^(?!({generate_le_regex(value)})).*)" +f"|(^0*"+ f"[0-9]{{{leng+1},}}$)"
print(generate_gt_regex(123))


['1', '2', '3']
122|(^0*1[0-1][0-9]{1}$)|(^0*12[0-2][0-9]{0}$)|(^0*[0-9]{1,2}$)
['1', '2', '4']
123|(^0*1[0-1][0-9]{1}$)|(^0*12[0-3][0-9]{0}$)|(^0*[0-9]{1,2}$)
['1', '2', '3']
^(?!(122|(^0*1[0-1][0-9]{1}$)|(^0*12[0-2][0-9]{0}$)|(^0*[0-9]{1,2}$))).*
['1', '2', '4']
(^(?!(123|(^0*1[0-1][0-9]{1}$)|(^0*12[0-3][0-9]{0}$)|(^0*[0-9]{1,2}$))).*)|(^0*[0-9]{4,}$)


In [134]:
def export_one_rule(component, is_not=False):
    elements = component.split(" ")
    operator = elements[0]

    # Hàm chuyển đổi giá trị thành số nguyên (nếu không được, trả về 0)
    def to_int(value):
        try:
            return int(value)
        except ValueError:
            return 0
    '''
    test:
    GET  hello world nGET
    GETn  hello world GET
    hello GET
    hello nGET
    hello .GET
    hello GET.
    '''
    match operator:

        case _ if operator.startswith("detect"):
            return None
        case "rx":
            return " ".join(elements[1:])
        case "beginsWith":
            return f"^{'(?!' if is_not else ''}{elements[1]}{')' if is_not else ''}.*"
        case "endsWith":
            return f"^(?!.*{elements[1]}$).+" if is_not else f".*{elements[1]}$"
        case "contains":
            return f"^(?!.*{elements[1]}).+" if is_not else f".*{elements[1]}.*"
        case "containsWord":
            return f"^(?!.*\\b{elements[1]}\\b).+" if is_not else f"\\b{elements[1]}\\b"
        case "eq":
            value = to_int(elements[1])
            # return f"^({value})$" if not is_not else f"^(?!({value})$).+"
            return generate_eq_regex(value)
        case "ge":
            value = to_int(elements[1])
            return generate_ge_regex(value)
        case "gt":
            value = to_int(elements[1])
            return generate_gt_regex(value)
        case "le":
            value = to_int(elements[1])
            return generate_le_regex(value)
        case "lt":
            value = to_int(elements[1])
            return generate_lt_regex(value)

def export_rule(mainRule):
    components = mainRule.split("@")
    main_sagan_rule = ""
    is_not = False
    for component in components:
        if component =="!":
            is_not = True
        else:
            if export_one_rule(component,is_not):
                main_sagan_rule += export_one_rule(component,is_not)
                is_not = False
    sagan_rule= "alert any $EXTERNAL_NET any -> $HOME_NET any ( pcre: \"/" + main_sagan_rule + "/\")"
    return sagan_rule


In [135]:
def extract_section(rule):
    first_quote = rule.find('"')
    if first_quote == -1:
        return None
    last_quote = rule.rfind('"')
    if last_quote == -1 or last_quote <= first_quote:
        return None
    return rule[first_quote + 1:last_quote]

def extract_id(line):
    match = re.search(r"id:(\d+)", line)
    return match.group(1) if match else None


In [136]:
def convert_file(input_file_path, output_file_path):
    """ Xử lý từng file rule và ghi kết quả ra file tương ứng """
    main_rules_dict = {}

    # Đọc file đầu vào
    with open(input_file_path, 'r', encoding="utf-8") as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        if lines[i].startswith("SecRule"):  # Nếu là rule
            rule = lines[i].strip()  # Lưu rule chính
            # Kiểm tra xem dòng tiếp theo có chứa 'id:' không
            if i + 1 < len(lines) and "id:" in lines[i + 1]:

                rule_id = extract_id(lines[i + 1])  # Lấy ID
                if rule_id:
                    main_rules_dict[rule_id] = extract_section(rule)  # Lưu vào dict
            i += 1  # Nhảy qua dòng tiếp theo (id:)
        i += 1  # Chuyển sang dòng kế tiếp

    # Ghi kết quả vào file mới
    with open(output_file_path, 'w', encoding="utf-8") as file:
        for rule_id, rule in main_rules_dict.items():
            if rule:
                sagan_rule = export_rule(rule)
                file.write(f"{rule_id}: {sagan_rule}\n")

    print(f"✅ Đã xử lý: {input_file_path} -> Lưu vào {output_file_path}")

In [137]:
def convert_mod_to_sagan(path, sagan_path):
    """ Đọc toàn bộ file trong thư mục path và xử lý từng file """
    # Đảm bảo thư mục kết quả tồn tại
    os.makedirs(sagan_path, exist_ok=True)

    for root, _, files in os.walk(path):
        for filename in files:
            input_file_path = os.path.join(root, filename)

            # Tạo đường dẫn thư mục tương ứng trong thư mục sagan_path
            relative_path = os.path.relpath(root, path)  # Đường dẫn tương đối
            output_dir = os.path.join(sagan_path, relative_path)
            os.makedirs(output_dir, exist_ok=True)

            # Tạo tên file kết quả
            output_file_path = os.path.join(output_dir, f"result_{filename}.txt")

            # Gọi hàm convert_file để xử lý file
            convert_file(input_file_path, output_file_path)

    print("✅ Completed processing all files!")


In [138]:
def main():
    path_data = "data"
    sagan_path = "result"
    convert_mod_to_sagan(path_data,sagan_path)

if __name__ == "__main__":
    main()

['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
✅ Đã xử lý: data\REQUEST-942-APPLICATION-ATTACK-SQLI.conf -> Lưu vào result\.\result_REQUEST-942-APPLICATION-ATTACK-SQLI.conf.txt
✅ Completed processing all files!
